![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace_in_Spark_NLP_CamemBertForZeroShotClassification.ipynb)

## Import CamemBertForZeroShotClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- This feature is only in `Spark NLP 4.4.1` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import CamemBERT models trained/fine-tuned for sequence classification via `CamembertForSequenceClassification` or `TFCamembertForSequenceClassification`. These models are usually under `Sequence Classification` category and have `camembert` in their labels
- Reference: [TFCamembertForSequenceClassification](https://huggingface.co/docs/transformers/model_doc/camembert#transformers.TFCamembertForSequenceClassification)
- Some [example models](https://huggingface.co/models?other=camembert&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- CamembertTokenizer requires the `SentencePiece` library, so we install that as well

In [ ]:
!pip install -q transformers tensorflow==2.11.0 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 111.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 111.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 73.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 3.19.6 which is incompatible.
google-cloud-iam 2.15.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.19.6 which is incompati

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [tblard/tf-allocine](https://huggingface.co/tblard/tf-allocine) model from HuggingFace as an example
- In addition to `TFCamembertForSequenceClassification` we also need to save the `CamembertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [ ]:
from transformers import TFCamembertForSequenceClassification
import tensorflow as tf


MODEL_NAME = 'mtheo/camembert-base-xnli'

try:
  model = TFCamembertForSequenceClassification.from_pretrained(MODEL_NAME)
except:
  model = TFCamembertForSequenceClassification.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask"),
          "token_type_ids": tf.TensorSpec((None, None), tf.int32, name="token_type_ids"),
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFCamembertForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFCamembertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFCamembertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFCamembertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertForSequenceClassification for predictions without further training.


Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {MODEL_NAME}

total 432424
-rw-r--r-- 1 root root       933 Jul 14 13:14 config.json
drwxr-xr-x 3 root root      4096 Jul 14 13:14 saved_model
-rw-r--r-- 1 root root 442786624 Jul 14 13:14 tf_model.h5


In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

total 8756
drwxr-xr-x 2 root root    4096 Jul 14 13:14 assets
-rw-r--r-- 1 root root      54 Jul 14 13:14 fingerprint.pb
-rw-r--r-- 1 root root  155351 Jul 14 13:14 keras_metadata.pb
-rw-r--r-- 1 root root 8797808 Jul 14 13:14 saved_model.pb
drwxr-xr-x 2 root root    4096 Jul 14 13:14 variables


We are using based model for the tokenizer because the model `mtheo/camembert-base-xnli` does not have sentencepiece

In [ ]:
from transformers import TFCamembertForSequenceClassification, CamembertTokenizer
import tensorflow as tf

# Try to load the tokenizer
try:
    tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
    print("Tokenizer loaded successfully!")
except OSError as e:
    print(f"Error loading tokenizer: {e}")

# Try to save the tokenizer
try:
    tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))
    print("Tokenizer saved successfully!")
except Exception as e:
    print(f"Error saving tokenizer: {e}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Tokenizer loaded successfully!
Tokenizer saved successfully!


In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

total 8756
drwxr-xr-x 2 root root    4096 Jul 14 13:14 assets
-rw-r--r-- 1 root root      54 Jul 14 13:14 fingerprint.pb
-rw-r--r-- 1 root root  155351 Jul 14 13:14 keras_metadata.pb
-rw-r--r-- 1 root root 8797808 Jul 14 13:14 saved_model.pb
drwxr-xr-x 2 root root    4096 Jul 14 13:14 variables


In [ ]:
!ls -l {MODEL_NAME}_tokenizer

total 804
-rw-r--r-- 1 root root     22 Jul 14 13:14 added_tokens.json
-rw-r--r-- 1 root root 810912 Jul 14 13:14 sentencepiece.bpe.model
-rw-r--r-- 1 root root    374 Jul 14 13:14 special_tokens_map.json
-rw-r--r-- 1 root root   1783 Jul 14 13:14 tokenizer_config.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `sentencepiece.bpe.model` file from the tokenizer
- all we need is to copy `sentencepiece.bpe.model` file into `saved_model/1/assets` which Spark NLP will look for
- in addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [ ]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

# let's copy sentencepiece.bpe.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/sentencepiece.bpe.model {asset_path}

In [ ]:
# get label2id dictionary
labels = model.config.label2id
# sort the dictionary based on the id
labels = sorted(labels, key=labels.get)

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

In [ ]:
!ls -lR {MODEL_NAME}/saved_model/1

mtheo/camembert-base-xnli/saved_model/1:
total 8756
drwxr-xr-x 2 root root    4096 Jul 14 13:14 assets
-rw-r--r-- 1 root root      54 Jul 14 13:14 fingerprint.pb
-rw-r--r-- 1 root root  155351 Jul 14 13:14 keras_metadata.pb
-rw-r--r-- 1 root root 8797808 Jul 14 13:14 saved_model.pb
drwxr-xr-x 2 root root    4096 Jul 14 13:14 variables

mtheo/camembert-base-xnli/saved_model/1/assets:
total 796
-rw-r--r-- 1 root root     32 Jul 14 13:14 labels.txt
-rw-r--r-- 1 root root 810912 Jul 14 13:14 sentencepiece.bpe.model

mtheo/camembert-base-xnli/saved_model/1/variables:
total 432308
-rw-r--r-- 1 root root 442663304 Jul 14 13:14 variables.data-00000-of-00001
-rw-r--r-- 1 root root     11573 Jul 14 13:14 variables.index


Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

## Import and Save CamemBertForZeroShotClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


- Let's use `loadSavedModel` functon in `CamemBertForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `CamemBertForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

zero_shot_classifier = CamemBertForZeroShotClassification\
  .loadSavedModel('{}/saved_model/1'.format(MODEL_NAME), spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
! rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your CamemBertForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 441436
-rw-r--r-- 1 root root 451208093 Jul 14 13:22 camembert_classification_tensorflow
-rw-r--r-- 1 root root    810912 Jul 14 13:22 camembert_spp
drwxr-xr-x 4 root root      4096 Jul 14 13:22 fields
drwxr-xr-x 2 root root      4096 Jul 14 13:22 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny CamemBertForZeroShotClassification model 😊

In [ ]:
zero_shot_classifier_loaded = CamemBertForZeroShotClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class") \
  .setCandidateLabels(["sport", "politique", "science"])

You can see what labels were used to train this model via `getClasses` function:

In [ ]:
# .getClasses was introduced in spark-nlp==3.4.0
zero_shot_classifier_loaded.getClasses()

['neutral', 'contradiction', 'entailment']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    zero_shot_classifier_loaded
])

# couple of simple examples
example = spark.createDataFrame([["Je m'appelle jean-baptiste et je vis à montréal"], ['george washington est allé à washington']]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "class.result").show()

+--------------------+-----------+
|                text|     result|
+--------------------+-----------+
|Je m'appelle jean...|    [sport]|
|george washington...|[politique]|
+--------------------+-----------+



That's it! You can now go wild and use hundreds of `CamemBertForZeroShotClassification` models from HuggingFace 🤗 in Spark NLP 🚀
